In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

In [2]:
!unzip -q dataset.zip

In [3]:
dataset = datasets.ImageFolder(
    'dataset',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

In [4]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - 90, 90])

In [5]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0,
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0,
)

In [6]:
model = models.resnet18(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 202MB/s]


In [7]:
model.fc = torch.nn.Linear(512, 2)

In [8]:
device = torch.device('cuda')
model = model.to(device)

In [9]:
NUM_EPOCHS = 30
BEST_MODEL_PATH = 'best_model_resnet18_second.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):

    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()

    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))

    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('%d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy

0: 0.888889
1: 0.866667
2: 0.944444
3: 0.988889
4: 0.966667
5: 0.977778
6: 0.977778
7: 1.000000
8: 0.888889
9: 1.000000
10: 0.966667
11: 0.955556
12: 0.988889
13: 1.000000
14: 0.977778
15: 0.977778
16: 0.977778
17: 0.988889
18: 0.988889
19: 1.000000
20: 0.977778
21: 0.988889
22: 0.988889
23: 0.988889
24: 0.977778
25: 0.977778
26: 0.977778
27: 1.000000
28: 0.988889
29: 0.988889


TEST MỘT ẢNH BẤT KỲ

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np
import time

device = torch.device('cuda')

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
model.load_state_dict(torch.load('best_model_resnet18_second.pth'))
model = model.to(device).eval().half()


mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()


def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

def update(change):
    image = change['new']
    x = preprocess(image)
    y = model(x)
    y = F.softmax(y, dim=1)

    prob_blocked = float(y.flatten()[0])
    print(f"Probability blocked: {prob_blocked:.2f}")

    if prob_blocked > 0.8:
        print("left")
    else:
        print("forward")

    time.sleep(0.001)


image_in = cv2.imread("test_block.jpeg")
# image_in = cv2.imread("f.jpg")
# image_in = cv2.imread("h.jpg")

image_in1 = cv2.cvtColor(image_in, cv2.COLOR_BGR2RGB)

resized = cv2.resize(image_in1, (224, 224))

cv2.imwrite("test.jpeg", resized)

image_final = cv2.imread("test.jpeg")

# cv2.imshow("Test",image_in)

update({'new': image_final})






Probability blocked: 1.00
left
